In [2]:
import numpy as np
import matplotlib.pyplot as plt
import visualization
import os
from gta_math import points_to_homo, ndc_to_view, construct_proj_matrix, view_to_world, construct_view_matrix
from visualization import load_depth, save_pointcloud_csv
import progressbar
from pointcloud_to_voxelmap import pointclouds_to_voxelmap
from joblib import Parallel, delayed
from configparser import ConfigParser
from PIL import Image
import pickle
from voxelmaps import camera_to_pointcloud, load_scene_db_data, get_main_image_name, scene_to_pointcloud, scene_to_voxelmap
import voxelmaps

In [2]:
ini_file = "gta-postprocessing.ini"
visualization.multi_page = False
visualization.ini_file = ini_file

conn = visualization.get_connection_pooled()
cur = conn.cursor()

CONFIG = ConfigParser()
CONFIG.read(ini_file)
in_directory = CONFIG["Images"]["Tiff"]
out_directory = CONFIG["Images"]["MlDatasetVoxel"]

In [3]:
run_id = 6

cur.execute("""SELECT DISTINCT scene_id \
  FROM snapshots \
  WHERE run_id = {} \
  """.format(run_id))

scenes = []
for row in cur:
    res = dict(row)
    scenes.append(res)

print('There are {} scenes'.format(len(scenes)))

There are 8438 scenes


### functions for each scene

In [4]:
# import time
voxelmaps.MAX_DISTANCE = 50 # in meters, meaning I care only for point up to 20 meters

def get_base_name(name):
    return os.path.basename(os.path.splitext(name)[0])


def convert_tiff(in_directory, out_directory, out_name, name):
    out_format = 'jpg'
    outfile = os.path.join(out_directory, "{}.{}".format(out_name, out_format))
    if os.path.exists(outfile):
        return
    try:
        infile = os.path.join(in_directory, name)
        im = Image.open(infile)
        im = im.convert(mode="RGB")
        # print("Generating new format for {} to new file {}".format(name, out_name))
        im.save(outfile)
    except OSError:
        # print("Skipping invalid file {}".format(name))
        return


def convert_scene_to_img_and_voxelmap(in_directory, out_directory, scene_id):
    if 'pbar' in globals() and 'counter' in globals():
        global counter
        counter += 1
        pbar.update(counter)

    # start = time.time()
    cameras = load_scene_db_data(scene_id)
    try:
        image_name = get_main_image_name(cameras)
        convert_tiff(in_directory, out_directory, image_name, image_name+'.tiff')
        outfile = os.path.join(out_directory, "{}.rick".format(image_name))

        if os.path.exists(outfile):
            return
        
        voxelmap = scene_to_voxelmap(cameras)
        with open(outfile, 'wb+') as f:
            pickle.dump(voxelmap, f)
    except NoMainImageException:
        print('No main image for scene {}, skipping.'.format(scene_id))


### actually running the extraction

In [ ]:
workers = 10

widgets = [progressbar.Percentage(), ' ', progressbar.Counter(), ' ', progressbar.Bar(), ' ',
           progressbar.FileTransferSpeed()]

pbar = progressbar.ProgressBar(widgets=widgets, maxval=len(scenes)).start()
counter = 0

Parallel(n_jobs=workers, backend='threading')(delayed(convert_scene_to_img_and_voxelmap)(in_directory, out_directory, i['scene_id']) for i in scenes)


  3% 278 |#                                                         | 274.6 B/s

No main image for scene 388309dc-568f-43ab-b2c1-86fb809cc570, skipping.


  5% 489 |###                                                       | 296.6 B/s

No main image for scene 592377fd-866a-4d81-bdb0-4e63ae584bab, skipping.
No main image for scene c70fe8d1-e628-483e-a27a-6a2c7b862857, skipping.


 16% 1351 |#########                                                | 329.5 B/s

...

 91% 7714 |####################################################     |   2.3 B/s

No main image for scene 6eb364df-255f-47dd-b45f-608a6aaac744, skipping.


 95% 8024 |######################################################   |   1.5 B/s

No main image for scene c0f6909d-532d-4c08-bca2-bc8a3eb7d534, skipping.


 95% 8057 |######################################################   |   1.4 B/s

### generating filenames list

In [ ]:
filenames = []
for i, (name, _) in enumerate(records):
    rgb_name = "{}.{}".format(get_base_name(name), rgb_format)
    rgb_name = os.path.relpath(os.path.join(out_directory, rgb_name), start=out_root_directory)
    rgb_name = rgb_name.replace('\\', '/')
    depth_name = "{}.{}".format(get_base_name(name), depth_format)
    depth_name = os.path.relpath(os.path.join(out_directory, depth_name), start=out_root_directory)
    depth_name = depth_name.replace('\\', '/')
    filenames.append([rgb_name, depth_name])

    # r je abych nemusel psát zpětná lomítka
with open(os.path.join(out_root_directory, 'whole-voxel-gta.csv'), mode='w+', newline='') as f:
    csv.writer(f).writerows(filenames)

random.shuffle(filenames)

train_ratio = 0.8
train_threshold = int(train_ratio * len(filenames))
train_filenames = filenames[:train_threshold]
test_filenames = filenames[train_threshold:]

with open(os.path.join(out_root_directory, 'train-voxel-gta.csv'), mode='w+', newline='') as f:
    csv.writer(f).writerows(train_filenames)
with open(os.path.join(out_root_directory, 'test-voxel-gta.csv'), mode='w+', newline='') as f:
    csv.writer(f).writerows(test_filenames)


### running extraction for one scene

In [ ]:
convert_scene_to_img_and_voxelmap(in_directory, out_directory, scenes[0]['scene_id'])

In [ ]:
# this is to visualize voxelmap as a pointcloud
def voxelmap_to_pointcloud_csv(path_in, path_out):
    with open(path_in, 'rb') as f:
        voxels, values, voxel_size = pickle.load(f)
    voxels = voxels[:, values >= 0]
    save_pointcloud_csv(voxels.T, path_out)


def scene_to_pointcloud_csv(scene_id, path_out):
    if 'pbar' in globals() and 'counter' in globals():
        global counter
        counter += 1
        pbar.update(counter)

    cameras = load_scene_db_data(scene_id)
    pointclouds, cam_positions = scene_to_pointcloud(cameras)
    big_pointcloud = np.zeros((3, 0))
    for pc in pointclouds:
        big_pointcloud = np.append(big_pointcloud, pc, axis=1)
    save_pointcloud_csv(big_pointcloud.T, path_out)


In [ ]:
voxelmap_to_pointcloud_csv(r'D:\ml-datasets-voxel\2018-03-07--15-19-54--505.rick', r'D:\ml-datasets-voxel\30m-2018-03-07--15-19-54--505.csv')
scene_to_pointcloud_csv(scenes[0]['scene_id'], r'D:\ml-datasets-voxel\pc-2018-03-07--15-19-54--505.csv')

 23% 1983 |#############                                            | 146.5 B/s

No main image for scene 08056d94-ab3d-42cd-8000-f9828444e783, skipping.

 24% 2049 |#############                                            |   5.8 B/s

No main image for scene 6f634b01-f5a0-4d65-b2fc-7df58b0d8904, skipping.

 24% 2086 |##############                                           |   3.4 B/sC:\ProgramData\Anaconda3\lib\site-packages\tifffile\tifffile.py:2642: RuntimeWarning: py_decodelzw encountered unexpected end of stream
  strip = decompress(strip)
 25% 2139 |##############                                           |   2.2 B/s